<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center>



<a target="_blank" href="https://colab.research.google.com/github/CienciaDeDatosEspacial/Operations-onGeoDF/blob/main/index.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Binary Spatial operations on  Geo Dataframes

This time the spatial operations will be applied when we have as input two GDFs. 

## Getting ready

The links to the our maps on GitHub are here:

In [ ]:
linkWorldMap="https://github.com/CienciaDeDatosEspacial/dataSets/raw/refs/heads/main/WORLD/worldMaps.gpkg"
linkBrazil="https://github.com/CienciaDeDatosEspacial/dataSets/raw/refs/heads/main/BRAZIL/brazil_5880.gpkg"
linkIndicators="https://github.com/CienciaDeDatosEspacial/dataSets/raw/refs/heads/main/WORLD/worldindicators.json"

Let me introduce another [data source on seaports](https://msi.nga.mil/Publications/WPI). Their locations in long/lat is stored in the file **seaports.csv** which we already have on GitHub:

In [ ]:
linkSeaPorts='https://github.com/CienciaDeDatosEspacial/dataSets/raw/refs/heads/main/WORLD/seaports.csv'

Let's get some maps:

In [ ]:
import geopandas as gpd

#world 
world_rivers=gpd.read_file(linkWorldMap,layer='rivers')
#brazil 
brazil5880=gpd.read_file(linkBrazil,layer='country')
airports_brazil5880=gpd.read_file(linkBrazil,layer='airports')
states_brazil5880=gpd.read_file(linkBrazil,layer='states')
municipalities_brazil5880=gpd.read_file(linkBrazil,layer='municipalities')
#some indicators
indicators=gpd.read_file(linkIndicators)

# the seaports
import pandas as pd 
infoseaports=pd.read_csv(linkSeaPorts)

In [ ]:
# the sesports data has too manny columns:
len(infoseaports.columns)

Let me keep some columns, and turn the DF into a GDF:

In [ ]:
#rename
infoseaports.rename(columns={'Main Port Name':'seaport_name','Country Code':'country_name'},inplace=True)

#keep few columns
infoseaports=infoseaports.loc[:,['seaport_name', 'country_name','Latitude', 'Longitude']]

#spatial points (unprojected)
seaports=gpd.GeoDataFrame(data=infoseaports.copy(),
                           geometry=gpd.points_from_xy(infoseaports.Longitude,
                                                       infoseaports.Latitude), 
                          crs=4326)# notice it is unprojected

# keep Brazil
seaports_bra=seaports[seaports['country_name']=='Brazil'].copy()

# reset indexes
seaports_bra.reset_index(drop=True, inplace=True)

# reprojecting
seaports_brazil5880=seaports_bra.to_crs(5880) # projected crs

**Before proceeding**, let me compute some GDFs we created the previous session:


* The centroid of Brazil

In [ ]:
brazil5880_cen=brazil5880.centroid

- Large Brazilian airports

In [ ]:
large_airports=airports_brazil5880.query("airport_type=='large_airport'")

- Amazon Rivers

In [ ]:
AmazonSystem=world_rivers.query("SYSTEM=='Amazon'")
AmazonSystem_5880=AmazonSystem.to_crs(5880)

- The states in the East, West, South, and North

In [ ]:
mid_x,mid_y=brazil5880_cen.x[0],brazil5880_cen.y[0]

# the north
N_brazil=states_brazil5880.cx[:,mid_y:]
# the south
S_brazil=states_brazil5880.cx[:,:mid_y]
# the west
W_brazil=states_brazil5880.cx[:mid_x,:]
# the east
E_brazil=states_brazil5880.cx[mid_x:,:]

- The mean fragility by region of the world

In [ ]:
indicatorsByRegion=indicators.dissolve(
    by="region", #groupby()
    aggfunc={"fragility": "mean"}, #agg()
    )

In [ ]:
# You may need to install if using Colab
# !pip install mapclassify


_____________



## I. Distance

Distance is a key binary operation as so many practical policy matters depend on knowing distances between objects in space.

Any pair of rightly projected GDFs have a distance between them. Below we can make  query using distances:

> Which are the airports whose distance to Brazil centroid is > 2500000?

In [ ]:
# this is the centroid we have:
brazil5880_cen

Then,

In [ ]:
airports_brazil5880[airports_brazil5880.distance(brazil5880_cen[0]) > 2500000]

The results can be confirmed visually:

In [ ]:
base=airports_brazil5880[airports_brazil5880.distance(brazil5880_cen.iloc[0]) > 2500000].plot(marker='+',markersize=100)
airports_brazil5880.plot(ax=base,color='grey', markersize=0.1)
brazil5880_cen.plot(ax=base,color='red')

Let me review how distances work between different kinds of geometries:

### a.  Distance between points

We have these points:


In [ ]:
seaports_brazil5880.head()

In [ ]:
large_airports.head()

For results of distance operation to be more insightful, we may do this:

In [ ]:
# index will be names of airports/seaports instead of numbers
large_airports.set_index('airport_name',inplace=True)
seaports_brazil5880.set_index('seaport_name',inplace=True)

> The distance from each airport to the "Dtse / Gegua Oil Terminal"

In [ ]:
large_airports.distance(seaports_brazil5880.geometry.iloc[0])

What about computing...

> All the distances between large aiports and seaports (in km)

In [ ]:
# apply creates a LOOP, computes distances from each seaport to all large airports
seaports_brazil5880.geometry.apply\
(lambda seaport: large_airports.geometry.distance(seaport)/1000)

If we save the matrix...

In [ ]:
D_Matrix_sea_air=seaports_brazil5880.geometry.apply \
                (lambda seaport: large_airports.geometry.distance(seaport)/1000)

We can compute some distance stats from there:

In [ ]:
Stat_sea_air=pd.DataFrame()
Stat_sea_air['mean']=D_Matrix_sea_air.mean(axis=1) # mean D to all airports
Stat_sea_air['min']=D_Matrix_sea_air.min(axis=1)# min D to all airports
Stat_sea_air['max']=D_Matrix_sea_air.max(axis=1)# max D to all airports

# see some
Stat_sea_air.head(10)

Of course, the idmax and idmin could come in handy:

In [ ]:
# farthest airport to each seaport
D_Matrix_sea_air.idxmax(axis=1).head()

In [ ]:
# farthest seaport to each airport
D_Matrix_sea_air.idxmax(axis=0).head()

In [ ]:
# closest airport to each seaport
D_Matrix_sea_air.idxmin(axis=1).head()

In [ ]:
# closest seaport to each airport
D_Matrix_sea_air.idxmin(axis=0).head()

### b. Distance between line and point

Once we know understand how **distance** and idxmin/idxmax work, we can feel comfortable in this stage. 

Let's use these rivers from before:

In [ ]:
AmazonSystem_5880.set_index("RIVER",inplace=True)
AmazonSystem_5880

Then,
> Distance from river Tapajos to Guarulhos airport

In [ ]:
airName='Guarulhos - Governador André Franco Montoro International Airport'
rivName='Tapajos'
AmazonSystem_5880.geometry.loc[rivName].distance(large_airports.geometry.loc[airName])/1000

We can compute the distance matrix now:

In [ ]:
D_Matrix_amazRivs_air=AmazonSystem_5880.geometry.apply \
                (lambda river: large_airports.geometry.distance(river)/1000)

In [ ]:
Stat_amz_air=pd.DataFrame()
Stat_amz_air['mean']=D_Matrix_amazRivs_air.mean(axis=1) # mean D to all airports
Stat_amz_air['min']=D_Matrix_amazRivs_air.min(axis=1)# min D to all airports
Stat_amz_air['max']=D_Matrix_amazRivs_air.max(axis=1)# max D to all airports

# see some
Stat_amz_air.head(10)

In [ ]:
# closest river to each airport
D_Matrix_amazRivs_air.idxmin(axis=0).head()

In [ ]:
# farthest river to each airport
D_Matrix_amazRivs_air.idxmax(axis=0).head()

### c. Between Polygon and Point

Let me re use the world rivers to get the rivers in a couple of systems:

In [ ]:
river_systems=world_rivers.query("SYSTEM in ['Amazon','Parana']")
river_systems

Let me combine per system:

In [ ]:
ama_para=river_systems.dissolve(by='SYSTEM')
ama_para.drop(columns='RIVER',inplace=True)
ama_para

We still have lines:

In [ ]:
ama_para.plot(cmap='viridis')

But we will have polygons after this:

In [ ]:
ama_para.convex_hull.plot(cmap='viridis')

As we have a geoseries of two geometries...

In [ ]:
ama_para.convex_hull,type(ama_para.convex_hull)

Let's turn that into a GDF:

In [ ]:
ama_para_hulls=ama_para.convex_hull.to_frame()
ama_para_hulls.rename(columns={0:'geometry'},inplace=True)
ama_para_hulls=ama_para_hulls.set_geometry('geometry')
ama_para_hulls.crs="EPSG:5880"

#voila
ama_para_hulls

And now, the distance matrix:

In [ ]:
D_Matrix_SYSHulls_air=ama_para_hulls.geometry.apply \
                (lambda system: large_airports.geometry.distance(system)/1000)
D_Matrix_SYSHulls_air

From here, you can compute distances between other kinds of geometries.

## II. Clipping

Clipping uses a GDF geometry as a MASK to cut another GDF which suppossedly is bigger and needes to be clipped.

Pay attention to the world rivers again:

In [ ]:
world_rivers

As you see, this GDF has no Country column. But since it has geometry, you can keep the rivers, or their sections, that serve a country:

In [ ]:
rivers_brazil5880 = gpd.clip(gdf=world_rivers.to_crs(5880),
                             mask=brazil5880)

Then, you can plot the clipped version:

In [ ]:
base = brazil5880.plot(facecolor="greenyellow", edgecolor='black', linewidth=0.4,figsize=(5,5))
rivers_brazil5880.plot(edgecolor='blue', linewidth=0.5,
                    ax=base)

We can create our own mask for clipping:

Let me get the **bounding box** of the map (the smallest possible rectangle that completely encloses a geometric shape or set of shapes):

In [ ]:
brazil5880.total_bounds #[minx, miny, maxx, maxy]

In [ ]:
# saving the output
minx, miny, maxx, maxy=brazil5880.total_bounds
minx, miny, maxx, maxy

I will combine those coordinates with the _centroid_ (mid_x,mid_y)to create a BOX of the north and south of Brazil:

In [ ]:
north_mask = [minx, mid_y, maxx, maxy]
south_mask = [minx, minx, maxx, mid_y]

# split Brazil
states_brazil5880.clip(north_mask).plot(edgecolor="yellow")

In [ ]:
states_brazil5880.clip(south_mask).plot(edgecolor="yellow")

As you see, with clip we can cut polygons (not respecting the borders).

## III. Spatial Joins

We’re familiar with **merging**, which joins tables using common keys. Spatial joins, by contrast, rely solely on **geometry columns** to perform various types of filtering. 

Let me use the brazilian large airports and states:

In [ ]:
large_airports.head()

...and:

In [ ]:
states_brazil5880.head()

In [ ]:
# as before
states_brazil5880.set_index('state_name',inplace=True)

### a. Within

Let's ask:
> The large airports whose geometries are within the borders of a state in Brazil.

In [ ]:
airports_within_states = gpd.sjoin(
    large_airports,         # LEFT: airports we want to filter/keep
    states_brazil5880,      # RIGHT: spatial boundaries to check against
    how='inner',            # return geometries that match in both LEFT/RIGHT (jointype)
    predicate='within'      # spatial condition: LEFT geometry within RIGHT geometry
)

# these are:
airports_within_states

We just performed a point-to-polygon spatial join.
Notice that the result preserves the original geometries from the LEFT GeoDataFrame — specifically, only those features whose spatial relationship satisfied both the predicate (e.g., 'within') and the join type ('inner').
The non-geometric attributes (columns) from the RIGHT GeoDataFrame are joined to the matching rows.

### b. Contains

Importantly, if the LEFT GeoDataFrame contains polygons and the RIGHT contains points (a polygon-to-point join), you’ll typically need to use a different predicate — such as 'contains' — to express the spatial relationship correctly.

> Brazilian states that house large airports

In [ ]:
states_containing_LargeAirports = gpd.sjoin(states_brazil5880,large_airports,how='inner',
                                            predicate='contains')

states_containing_LargeAirports

### c. Intersects

'Contains' is literally strict: Any airport located exactly on a state boundary — whether due to data precision, snapping, or real geography — will be excluded, even if it’s “practically” inside the state. More flexibility is achieved with **intersects**.

> Large airports whose location is anywhere belonging to a particular state

In [ ]:
## Intersects needs at least a common point between both GeoDFs. 
gpd.sjoin(states_brazil5880,large_airports,
          how='inner', predicate='intersects')

### d. Touches

We also have 'touches', a more stringent predicate than 'intersects'. It returns geometries that:
 - Share a border (for polygons or lines), or
 - Contact at exactly one point (for points or endpoints).

> Which states are neighbors of 'BAHIA", including BAHIA

In [ ]:
# Neighbors of Bahia?
gpd.sjoin(N_brazil.loc[N_brazil.state_name=='Bahia',:],N_brazil,how='inner', predicate='intersects').shape

That is, Bahia seems to share borders with 5 states:

In [ ]:
base=gpd.sjoin(N_brazil,N_brazil.loc[N_brazil.state_name=='Bahia',:],
               how='inner', 
               predicate='intersects').plot(color='yellow',edgecolor='red')
N_brazil.loc[N_brazil.state_name=='Bahia',:].plot(ax=base, color='red')

However, because many free GeoDataFrames — especially those sourced as Shapefiles — contain topological imperfections like gaps, overlaps, or misaligned vertices, 'touches' often fails to detect what should be adjacent features. Ironically, this “failure” can be useful: 'touches' acts as a diagnostic tool — highlighting where boundaries are not perfectly aligned.

In [ ]:
gpd.sjoin(N_brazil.loc[N_brazil.state_name=='Bahia',:],N_brazil,how='inner', predicate='touches').shape

See the neighbor that disappears:

In [ ]:
base=gpd.sjoin(N_brazil,N_brazil.loc[N_brazil.state_name=='Bahia',:],
               how='inner', 
               predicate='touches').plot(color='yellow',edgecolor='red')
N_brazil.loc[N_brazil.state_name=='Bahia',:].plot(ax=base, color='red')

### e. Crosses

When we have **lines**, we may need **crosses**. Let me subset our rivers:

In [ ]:
amazonSystem=rivers_brazil5880[rivers_brazil5880.SYSTEM=='Amazon']
amazonSystem.set_index('RIVER',inplace=True)

Then,
> Which rivers from the Amazon system are intersecting states?

In [ ]:
gpd.sjoin(amazonSystem,states_brazil5880,how='inner', predicate='intersects')

A count of the result:

In [ ]:
gpd.sjoin(amazonSystem,states_brazil5880,how='inner', predicate='intersects').shape

Alternatively,

> Which rivers from the Amazon system are crossing states?

In [ ]:
gpd.sjoin(amazonSystem,states_brazil5880,how='inner', predicate='crosses')

You got one less:

In [ ]:
gpd.sjoin(amazonSystem,states_brazil5880,how='inner', predicate='crosses').shape

Again, _intersects_ means both geometries have some 'space' in common. But **crosses** is an intersection that has to cross the spatial object. From the result above, there is one river that shares space with the state, but is not crossing its border:

In [ ]:
# Get intersects result
intersects_result = gpd.sjoin(amazonSystem,states_brazil5880, how='inner', predicate='intersects')
intersects_result

In [ ]:
# Get crosses result
crosses_result = gpd.sjoin(amazonSystem,states_brazil5880, how='inner', predicate='crosses')
crosses_result

In [ ]:
river_notCrossing=list(set(intersects_result.index)-set(crosses_result.index))
river_notCrossing

In [ ]:
# Find indexes/columns
state_notCrossed=intersects_result.loc[river_notCrossing,'state_name'].to_list()
state_notCrossed

Now we know the river that is not crossing an state, and the name of that state.

In [ ]:
base=states_brazil5880.loc[state_notCrossed,:].plot(color='w',edgecolor='k',figsize=(12, 8))
amazonSystem.plot(ax=base)
amazonSystem.loc[river_notCrossing,:].plot(color='red',ax=base)


## IV. Spatial Overlay

As the name implies, you need two inputs. We might need to create or find some geometries from the geometries we already have. Using a set theory approach, we will see the use of _intersection_, _union_, _difference_, and _symmetric difference_.
Let's remember these results:

In [ ]:
N_brazil

In [ ]:
S_brazil

Let me plot both of them:

In [ ]:
base= N_brazil.plot(facecolor='black', edgecolor='white',linewidth=0.2, alpha=0.6)
S_brazil.plot(facecolor='white', edgecolor='black',linewidth=0.2,ax=base, alpha=0.6)

The grey area reminds you that the coordinates we used to split the states did not give us a clean cut. Here you see the states in common:

In [ ]:
set(S_brazil.state_name) & set(N_brazil.state_name)

The same happened in East vs West:

In [ ]:
set(E_brazil.state_name) & set(W_brazil.state_name)

In [ ]:
# visualizing
base= E_brazil.plot(facecolor='black', edgecolor='white',linewidth=0.2, alpha=0.6)
W_brazil.plot(facecolor='white', edgecolor='black',linewidth=0.2,ax=base, alpha=0.6)

Let's play with these GDFs, keep in mind the position of the GDFs:

```
GDF_left.overlay(GDF_right, ...)
```

### a. Intersection

We keep what is common between _left_ and _right_ GDFs :

In [ ]:
NS_brazil=N_brazil.overlay(S_brazil, how="intersection",keep_geom_type=True)
# see results
NS_brazil

Notice we got more rows than when we did this operation:

```
set(S_brazil.state_name) & set(N_brazil.state_name)
```
We have three more polygons:

In [ ]:
NS_brazil[NS_brazil.state_name_1!= NS_brazil.state_name_2]

In fact, we are NOT intersecting state names, we are intersecting geometries. Then, the input maps have some topological issues.

This is the amount of area that is in fact a topological problem:

In [ ]:
NS_brazil[NS_brazil.state_name_1!= NS_brazil.state_name_2].geometry.area.sum()

A way to measure the share of the low quality:

In [ ]:
NS_brazil[NS_brazil.state_name_1!= NS_brazil.state_name_2].geometry.area.sum()/  \
NS_brazil[NS_brazil.state_name_1== NS_brazil.state_name_2].geometry.area.sum() #continues from above

So, spatial overlay operations do their best to give you true results; but unfortunately, as the quality of the sources is not perfect, you may get messy results. It is our job to detect and make decisions. Let's keep two GDFs, one with the unperfect result, and another with a 'valid' output.

In [ ]:
NS_brazil_messy=NS_brazil.copy()
NS_brazil=NS_brazil[NS_brazil.state_name_1== NS_brazil.state_name_2]

This should be what we expected to see:

In [ ]:
NS_brazil

The clean data has minor things to improve, delete redundant columns, rename columns, and reset the index so they are a correlative sequence. 

In [ ]:
# avoid redundancy
keep=['state_name_1','state_code_1','geometry']
NS_brazil=NS_brazil.loc[:,keep]
NS_brazil.rename(columns={'state_name_1':'state_name','state_code_1':'state_code'},inplace=True)

# reset for correlative sequence
NS_brazil.reset_index(drop=True, inplace=True)

Based on the previous case, we may expect a similar situation here:

In [ ]:
# keeping the overlay
WE_brazil=W_brazil.overlay(E_brazil, how="intersection",keep_geom_type=True)
WE_brazil[WE_brazil.state_name_1!= WE_brazil.state_name_2]

Let's do the same as before:

In [ ]:
WE_brazil_messy=WE_brazil.copy()
WE_brazil=WE_brazil[WE_brazil.state_name_1== WE_brazil.state_name_2]

keep=['state_name_1','state_code_1','geometry']
WE_brazil=WE_brazil.loc[:,keep]
WE_brazil.rename(columns={'state_name_1':'state_name','state_code_1':'state_code'},inplace=True)
WE_brazil.reset_index(drop=True, inplace=True)

Check this last one:

In [ ]:
base=brazil5880.plot(facecolor='yellow')
WE_brazil.plot(ax=base)

### b. Union

Different from UNION_ALL (which acts as DISSOLVE), here we will combine the left and right GDFs. 

In [ ]:
NS_brazil.info()

In [ ]:
WE_brazil.info()

In [ ]:
# now
NS_brazil.overlay(WE_brazil,how="union",keep_geom_type=True)

As you see, geometries are fine, but missing values were created where no intersection exists. Notice this operation  identifies the intersection. 

Pandas has a _concat()_ function, which just pastes two tables, one of top of the other. Notice this will give you one more row (Mato Grosso appears twice) as nothing is done spite of geometries intersect.

In [ ]:
# appending
import pandas as pd

pd.concat([NS_brazil,WE_brazil],ignore_index=True)

Let me create an object to save the overlay result:

In [ ]:
MidBrazil=NS_brazil.overlay(WE_brazil,how="union",keep_geom_type=True).dissolve()
MidBrazil

In [ ]:
# some cleaning

MidBrazil['country']='Brazil'
MidBrazil['region']='center'
# reordering
MidBrazil=MidBrazil.loc[:,['country','region','geometry']]

MidBrazil

In [ ]:
# see it
base=brazil5880.plot(facecolor='yellow')
MidBrazil.plot(ax=base)

### c. Difference

Here, you keep what belongs to the left GDF that is not in the right GDF:

In [ ]:
# we keep nothern states that are not in the southern ones
N_brazil.overlay(S_brazil, how='difference')

In [ ]:
# using set operations:
set(N_brazil.state_name)- set(S_brazil.state_name)

We got a clean result (remember we are using the not messy GDFs). Let's plot this:

In [ ]:
base=N_brazil.plot(color='yellow', edgecolor='black',alpha=0.1)
N_brazil.overlay(S_brazil, how='difference').plot(ax=base)

Keep in mind that **difference** is not commutative:

In [ ]:
S_brazil.overlay(N_brazil, how='difference')

This a totally different result:

In [ ]:
base=N_brazil.plot(color='yellow', edgecolor='black',alpha=0.1)
S_brazil.overlay(N_brazil, how='difference').plot(ax=base)

### d. Symmetric Difference

Here, we keep what is not in the intersection but in both GDFs. Notice that this operation is commutative!

In [ ]:
N_brazil.overlay(S_brazil, how='symmetric_difference')

This operation gave a clean result again. Let's plot it:

In [ ]:
N_brazil.overlay(S_brazil, how='symmetric_difference').plot()


_____________


# Validity of Geometries

Remember this case from last class:

In [ ]:
# previously
indicatorsByRegion.plot(column =indicatorsByRegion.index,
                        edgecolor='white',
                        figsize=(15, 10))

Let's explore with these last operations what is the topological issue:

In [ ]:
china=indicators[indicators.Country.isin(['CHINA'])]
mongolia=indicators[indicators.Country.isin(['MONGOLIA'])]

china.overlay(mongolia, how='intersection',keep_geom_type=False).geometry

So, we have some really bad situation:

- There is an intersection between two countries, and there should be none.
- There intersection includes objects other than polygons: 'GEOMETRYCOLLECTION'

See:


In [ ]:
# Quick count of objects in the GeometryCollection
result_geom = china.overlay(mongolia, how='intersection',keep_geom_type=False).geometry.iloc[0]
if result_geom.geom_type == 'GeometryCollection':
    print(f"Objects in collection: {len(result_geom.geoms)}")
    from collections import Counter
    print(dict(Counter(g.geom_type for g in result_geom.geoms)))

In [ ]:
## see the intersection:
base=china.plot(color='lightgrey')
mongolia.plot(color='yellow',ax=base)
china.overlay(mongolia, how='intersection',keep_geom_type=False).plot(ax=base)

The solution to this, believe me, will not be trivial: the border is not continuous, and creating a 'new frontier' between China and Mongolia will demand more functions than the ones we have taught so far. Situations like this require smart decisions, like get a new map  with a [better quality](https://www.naturalearthdata.com/downloads/110m-cultural-vectors/).